In [1]:
# try to connect sql
import pymysql
import pandas as pd
import os
import csv

HOST = "127.0.0.1"
PORT = 3306
USER = 'root'
PASS = "12345678"
DB = "bookrs"

In [22]:
conn = pymysql.connect(
    host=HOST, 
    port=PORT, 
    user=USER,
    password=PASS,
    database=DB,
    charset="utf8")

cursor = conn.cursor()

cursor.execute('drop table if exists TEST')
# sql = """
# create table `test` (book_id INT AUTO_INCREMENT not null primary key, `best_book_id` INT,`work_id` INT,`books_count` INT,`isbn` VARCHAR(255),`isbn13` VARCHAR(255),`authors` VARCHAR(510),`original_publication_year` VARCHAR(255),`original_title` VARCHAR(255),`title` VARCHAR(255),`language_code` VARCHAR(255),`average_rating` FLOAT,`ratings_count` INT,`work_ratings_count` INT,`work_text_reviews_count` INT,`ratings_1` INT,`ratings_2` INT,`ratings_3` INT,`ratings_4` INT,`ratings_5` INT,`image_url` VARCHAR(255),`small_image_url` VARCHAR(255))
# """

sql = "create table `TEST` (id INT AUTO_INCREMENT not null primary key, `book_id` INT,`user_id` INT,`rating` INT)"
cursor.execute(sql)
cursor.close()

conn.close()

/Users/wujingyi/miniconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1051, "Unknown table 'bookrs.books'")
  result = self._query(query)


In [16]:
rating_df = pd.read_csv('./book-data/ratings.csv', header = 0)
rating_df.shape

(981756, 3)

In [64]:
user_df = rating_df.groupby('user_id')['user_id'].agg(['count']).reset_index()

In [67]:
# user_df[user_df['count']>30]
# user_df

In [68]:
rating_df

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
5,1,2077,4
6,1,2487,4
7,1,2900,5
8,1,3662,4
9,1,3922,5


In [49]:
class CsvToMysql(object):
    def __init__(self, hostname, port, user, passwd, db):
        self.dbname = db
        self.conn = pymysql.connect(host=hostname, port=port, user=user, passwd=passwd, db=db)
        self.cursor = self.conn.cursor()
 
 
    def read_csv(self,filename,indexcol=None):
        df = pd.read_csv(filename, keep_default_na=False, encoding='utf-8')
        try:
            df.drop(['id'],axis=1,inplace=True)
        except:
            pass
        
        table_name = '`'+os.path.split(filename)[-1].split('.')[0].replace(' ', '_') + '`'
        self.csv2mysql(db_name=self.dbname,table_name=table_name, df=df, col_index = indexcol)
 
    def read_dataframe(self,table_name, dataframe,indexcol=None):
        table_name = '`'+table_name+ '`'
        self.csv2mysql(db_name=self.dbname,table_name=table_name, df=dataframe, col_index = indexcol)
    
 
    def make_table_sql(self,df,col_index):
        columns = df.columns.tolist()
        types = df.ftypes
        make_table = []
        make_field = []
        if col_index == None:
            print('none index')
            make_table.append('id INT')
        for item in columns:
            if item ==col_index:
                item1 = item.replace(' ', '_').replace(':','')
            else:
                item1 = '`'+item.replace(' ', '_').replace(':','')+'`'
                
            if 'int' in types[item]:
                char = item1 + ' INT'
            elif 'float' in types[item]:
                char = item1 +' FLOAT'
            elif 'object' in types[item]:
                char = item1 +' VARCHAR(1000)'
            elif 'datetime' in types[item]:
                char = item1 + ' DATETIME'
            else:
                char = item1 + ' VARCHAR(400)'
            #char = item1 + ' VARCHAR(255)'
            make_table.append(char)
            make_field.append(item1)

#         return ','.join(make_table), ','.join(make_field)
        return make_table,make_field

    def csv2mysql(self,db_name,table_name,df, col_index):
        make_table, make_field = self.make_table_sql(df,col_index)

        field1 = ','.join(make_table[1:])
        field2 = ','.join(make_field)
        
        print("create table {} ({} AUTO_INCREMENT not null primary key, {})".format(table_name,make_table[0],field1))
        self.cursor.execute('drop table if exists {}'.format(table_name))
        self.cursor.execute("create table {} ({} AUTO_INCREMENT not null primary key,{})".format(table_name,make_table[0],field1))
        values = df.values.tolist()
        s = ','.join(['%s' for _ in range(len(df.columns))])
        try:
            print(len(values[0]),len(s.split(',')))
            print('insert into {}({}) values ({})'.format(table_name, field2, s), values[0])
#             if col_index == None:
#                 self.cursor.executemany('insert into {}({}) values ({})'.format(table_name, field2, s), values)
#             else:
            self.cursor.executemany('insert into {}({}) values ({})'.format(table_name, field2, s), values)
            
        except Exception as e:
            print("Exception",e)
        finally:
            self.conn.commit()

In [50]:
M = CsvToMysql(hostname=HOST, port=PORT, user=USER, passwd=PASS, db=DB)

In [66]:
# M.read_dataframe("users", user_df,indexcol="user_id")

create table `users` (user_id INT AUTO_INCREMENT not null primary key, `count` INT)
2 2
insert into `users`(user_id,`count`) values (%s,%s) [1, 3]


In [51]:
# M.read_dataframe("rating_record", rating_df)

none index
create table `rating_record` (id INT AUTO_INCREMENT not null primary key, `book_id` INT,`user_id` INT,`rating` INT)
3 3
insert into `rating_record`(`book_id`,`user_id`,`rating`) values (%s,%s,%s) [1, 314, 5]


In [52]:
# M.read_csv('./book-data/books.csv','book_id')

create table `books` (book_id INT AUTO_INCREMENT not null primary key, `best_book_id` INT,`work_id` INT,`books_count` INT,`isbn` VARCHAR(1000),`isbn13` VARCHAR(1000),`authors` VARCHAR(1000),`original_publication_year` VARCHAR(1000),`original_title` VARCHAR(1000),`title` VARCHAR(1000),`language_code` VARCHAR(1000),`average_rating` FLOAT,`ratings_count` INT,`work_ratings_count` INT,`work_text_reviews_count` INT,`ratings_1` INT,`ratings_2` INT,`ratings_3` INT,`ratings_4` INT,`ratings_5` INT,`image_url` VARCHAR(1000),`small_image_url` VARCHAR(1000))
22 22
insert into `books`(book_id,`best_book_id`,`work_id`,`books_count`,`isbn`,`isbn13`,`authors`,`original_publication_year`,`original_title`,`title`,`language_code`,`average_rating`,`ratings_count`,`work_ratings_count`,`work_text_reviews_count`,`ratings_1`,`ratings_2`,`ratings_3`,`ratings_4`,`ratings_5`,`image_url`,`small_image_url`) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) [2767052, 2767052, 2792775, 272, '